# Portfolio Optimization Project

### CS/ECE/ISyE 524 — Introduction to Optimization — Fall 2024

### Project Members: Pawin Linmaneechote, Vincent Philavong, Jiajie Yao

## Table of Contents
1. Introduction  
    1.1. Project Motivation  
    1.2. Data Processing  
2. Mathematical Model
    2.1 Mean Variance Model
    2.2 Mean Variance Model + Ridge Regression
    2.3 Sharpe SOCP Model
3. Solution
4. Results and Discussion
5. Conclusion
6. Appendix


# 1. Introduction
## 1.1. Project Motivation
Investing is about making smart decisions to balance the trade-offs between risk and reward. In this project, we explore portfolio optimization by analyzing financial data from Yahoo Finance. We’ve chosen five stocks from key sectors - Technology, Financials, Healthcare, Communication Services, and Industrials - to build a diversified portfolio. This selection spreads risk across industries while aiming to capture each sector’s growth potential. The list of the stock sectors and the individual stocks can be seen below.
- Technology
    - Apple (AAPL)
    - Uber (UBER)
    - Microsoft (MSFT)
    - NVIDIA (NVDA)
    - Advanced Micro Devices (AMD)
- Financial
    - Discover (DFS)
    - JP Morgan (๋JPM)
    - Synchrony (SYF)
    - American Express (AXP)
    - Goldman Sachs (GS)
- Health Care
    - Pfizer (PFE)
    - Johnson & Johnson (JNJ)
    - Merck Sharp & Dohme (MRK)
    - United Health Group (UNH)
    - ABBVie (ABBV)
- Communication Services
    - Meta (META)
    - Google (GOOG)
    - Netflix (NFLX)
    - T-Mobile (TMUS)
    - The Walt Disney Services (DIS)
- Industrials
    - Caterpillar (CAT)
    - RTX Coorporation (RTX)
    - The Boeing Company (BA)
    - Union Pacific Corperation (UNP)
    - Eaton Corporation PLC (ETN)

Our project focuses on optimizing three models. The first serves as our baseline, addressing the challenge of balancing high returns with low risk by analyzing the performance and relationships between the selected stocks. The second builds on the baseline by incorporating Ridge Regularization, which we believe will further enhance diversification within the portfolio. For the third model, we introduce a Sharpe SOCP model, focusing on maximizing returns while minimizing risk more effectively.

Portfolio optimization has been a topic of interest for investors and researchers for decades. It’s rooted in the uncertainty of financial markets, where both short-term gains and long-term stability needed careful evaluation. The economist, Harry Markowitz, revolutionized this field in 1952 with his paper Portfolio Selection, introducing diversification as a way to reduce risk (Team Investopedia, 2023). His work laid the groundwork for modern portfolio theory (MPT) and earned him a Nobel Prize. Yet, as markets continue to evolve, ongoing research is critical to adapting strategies for changing conditions like volatility, economic shifts, and new investor behaviors.

This report is structured into four sections: the Mathematical Model, where we define the decision variables, constraints, and objective function; the Solution, where we implement the model in Julia; Results and Discussion, where we present and interpret the outcomes; and the Conclusion, summarizing our findings and proposing future directions.

## 1.2 Data Preparation

In [ ]:
using CSV, DataFrames, Dates, Statistics, JuMP, Ipopt, PyPlot, LinearAlgebra

# Define the directory containing the stock CSV files
directory_path = "data_files/"

# Initialize an empty DataFrame to hold daily returns
returns_df = DataFrame(Date = Date[])

# Read each stock CSV file and compute daily returns
for file in readdir(directory_path)
    if endswith(file, ".csv")
        file_path = joinpath(directory_path, file)
        stock_df = CSV.read(file_path, DataFrame)

        # Ensure the Date column is of Date type
        stock_df.Date = Date.(string.(stock_df.Date), "yyyy-mm-dd")
        sort!(stock_df, :Date)

        stock_name = splitext(basename(file))[1]

        # Calculate daily logarithmic returns
        stock_df[!, :DailyReturn] = [missing; diff(log.(stock_df.Close))]

        # Create a DataFrame with Date and DailyReturn
        stock_returns = stock_df[:, [:Date, :DailyReturn]]
        rename!(stock_returns, :DailyReturn => stock_name)

        # Merge with the returns_df
        if nrow(returns_df) == 0
            returns_df = stock_returns
        else
            returns_df = outerjoin(returns_df, stock_returns, on = :Date)
        end
    end
end

# Remove rows with missing values
complete_cases = dropmissing(returns_df, disallowmissing=true)

# --- Extract Asset Names and Prepare Data ---

# Extract asset names excluding the Date column
# Check the type of column names
col_names = names(complete_cases)
println("Column names:", col_names)
println("Type of column names:", typeof.(col_names))

# Adjust based on the type
if eltype(col_names) == String
    asset_names = setdiff(col_names, ["Date"])
elseif eltype(col_names) == Symbol
    asset_names = setdiff(col_names, [:Date])
else
    error("Unexpected column name types.")
end

# Convert asset_names to strings if necessary
asset_names = string.(asset_names)

# Verify lengths
println("Number of asset names:", length(asset_names))

# Extract the matrix of returns (excluding the Date column)
returns_matrix = Matrix(complete_cases[:, asset_names])

# --- Compute Expected Returns and Covariance Matrix ---

trading_days_per_year = 252  # Number of trading days in a year

# Compute expected daily returns
μ_daily = mean(returns_matrix, dims=1)
μ_daily = vec(μ_daily)  # Convert to a vector

# Annualize expected returns (using compounding)
μ = (1 .+ μ_daily) .^ trading_days_per_year .- 1

# Verify length of μ
println("Length of μ:", length(μ))

# Compute daily covariance matrix
Σ_daily = cov(returns_matrix, dims=1)

# Annualize covariance matrix
Σ = Σ_daily * trading_days_per_year

# Verify size of Σ
println("Size of Σ:", size(Σ))

# Number of assets
n_assets = length(μ)
println("Number of assets (n_assets):", n_assets)



# 2. Mathematical Model

#### Mean Variance Model

In [ ]:
# --- Portfolio Optimization (Mean Varaiance Model)

λ = 0     # Risk aversion parameter

# Create optimization model
m = Model(Ipopt.Optimizer)

@variable(m, x[1:n_assets] >= 0)
@constraint(m, sum(x) == 1)

@objective(m, Max, μ' * x - λ * (x' * Σ * x))

# Solve the model
optimize!(m)

# Check solution status
status = termination_status(m)
display(status)

if status == MOI.OPTIMAL || status == MOI.LOCALLY_SOLVED
    println("The model is feasible.")

    # Extract optimal portfolio weights
    portfolio_weights = value.(x)

    # Compute portfolio return and risk
    portfolio_return = μ' * portfolio_weights  # Annual expected return
    portfolio_risk = sqrt(portfolio_weights' * Σ * portfolio_weights)  # Annual standard deviation

    # --- Plotting ---
    figure(figsize=(12, 6))
    bar(1:n_assets, portfolio_weights, color="blue", alpha=0.7)
    xticks(1:n_assets, asset_names, rotation=90)
    xlabel("Assets")
    ylabel("Portfolio Weight")
    title("Optimal Portfolio Weights for R_min = $R_min")
    grid(true)
    tight_layout()
    show()

    # --- Display Portfolio Metrics ---
    println("Portfolio Expected Annual Return: $portfolio_return")
    println("Portfolio Risk (Annual Standard Deviation): $portfolio_risk")
else
    println("The model is not feasible. Status: $status")
end

#### Mean Variance Model + RIDGE Regression

In [ ]:
# --- Portfolio Optimization (Mean Varaiance Model + RIDGE Regression)

λ = 0     # Risk aversion parameter
γ = 1000        # Diversification parameter

# Create optimization model
m = Model(Ipopt.Optimizer)

@variable(m, x[1:n_assets] >= 0)
@constraint(m, sum(x) == 1)
#@constraint(m, [i in 1:n_assets], x[i] <= 0.15)

@objective(m, Max, μ' * x - λ * (x' * Σ * x) - γ * sum(x[i]^2 for i in 1:n_assets))

# Solve the model
optimize!(m)

# Check solution status
status = termination_status(m)
display(status)

if status == MOI.OPTIMAL || status == MOI.LOCALLY_SOLVED
    println("The model is feasible.")

    # Extract optimal portfolio weights
    portfolio_weights = value.(x)

    # Compute portfolio return and risk
    portfolio_return = μ' * portfolio_weights  # Annual expected return
    portfolio_risk = sqrt(portfolio_weights' * Σ * portfolio_weights)  # Annual standard deviation

    # --- Plotting ---
    figure(figsize=(12, 6))
    bar(1:n_assets, portfolio_weights, color="blue", alpha=0.7)
    xticks(1:n_assets, asset_names, rotation=90)
    xlabel("Assets")
    ylabel("Portfolio Weight")
    title("Optimal Portfolio Weights")
    grid(true)
    tight_layout()
    show()

    # --- Display Portfolio Metrics ---
    println("Portfolio Expected Annual Return: $portfolio_return")
    println("Portfolio Risk (Annual Standard Deviation): $portfolio_risk")
else
    println("The model is not feasible. Status: $status")
end

LOCALLY_SOLVED::TerminationStatusCode = 4

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       25
Number of nonzeros in inequality constraint Jacobian.:       25
Number of nonzeros in Lagrangian Hessian.............:       25

Total number of variables............................:       25
                     variables with only lower bounds:       25
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -2.4557224e+00 7.50e-01 9.66e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

sys:1: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown


#### Sharpe Model SOCP

In [ ]:
# --- Portfolio Optimization (Sharpe Model SOCP) 

# Define your data
n_assets = length(μ)
r_f = 0.02    # Risk-free rate
R_min = 0.05    # Minimum expected return (if needed)

# Create optimization model
m = Model(Ipopt.Optimizer)

# Define variables
@variable(m, w[1:n_assets] >= 0)   # Portfolio weights (non-negative)
@variable(m, t >= 0)               # Scalar variable t

# Objective function: maximize t * (μ' * w - r_f)
@objective(m, Max, t * (dot(μ, w) - r_f))

#@constraint(m, [i in 1:n_assets], w[i] <= 0.5)

# Constraints
@constraint(m, sum(w) == 1)        # Budget constraint

# Nonlinear constraint: t^2 * w' * Σ * w <= 1
@constraint(m, t^2 * w' * Σ * w  .<= 1)

# Solve the model
optimize!(m)

# Check the termination status
status = termination_status(m)
display(status)

if status == MOI.OPTIMAL || status == MOI.LOCALLY_SOLVED
    println("The model is feasible.")

    # Extract optimal portfolio weights
    portfolio_weights = value.(w)

    # Compute portfolio return and risk
    portfolio_return = dot(μ, portfolio_weights)  # Annual expected return
    portfolio_risk = sqrt(dot(portfolio_weights, Σ * portfolio_weights))  # Annual standard deviation

    # --- Plotting ---
    figure(figsize=(12, 6))

    # Plotting the bar chart
    bar(1:n_assets, portfolio_weights, color="blue", alpha=0.7)

    # Set x-axis labels and rotation
    xticks(1:n_assets, asset_names, rotation=90)

    # Add labels and title
    xlabel("Assets")
    ylabel("Portfolio Weight")
    title("Optimal Portfolio Weights")

    # Add grid and adjust layout
    grid(true)
    tight_layout()

    # Display the plot
    show()

    # --- Display Portfolio Metrics ---
    println("Portfolio Expected Annual Return: $(round(portfolio_return * 100, digits=2))%")
    println("Portfolio Risk (Annual Standard Deviation): $(round(portfolio_risk * 100, digits=2))%")
    println("Maximum Sharpe Ratio: $(round(objective_value(m), digits=4))")
else
    println("The model is not feasible. Status: $status")
end


LOCALLY_SOLVED::TerminationStatusCode = 4

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       25
Number of nonzeros in inequality constraint Jacobian.:       26
Number of nonzeros in Lagrangian Hessian.............:      376

Total number of variables............................:       26
                     variables with only lower bounds:       26
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.4272542e-04 7.50e-01 1.02e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

sys:1: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown



Portfolio Risk (Annual Standard Deviation): 36.28%
Maximum Sharpe Ratio: 1.7603


# 3. Solutions

In [18]:
# Calculate total weight for each sector
sector_weights = Dict()
for (i, sector) in enumerate(final_df.StockSector)
    weight = value.(x)[i]
    # Ensure weights are non-negative and handle very small numbers
    weight = max(0, weight)
    if weight > 1e-10  # Only add if weight is significant
        sector_weights[sector] = get(sector_weights, sector, 0) + weight
    end
end

println("Sector weights: ", sector_weights)
println("Total weight: ", sum(values(sector_weights)))

figure(figsize=(10, 8))
pie(collect(values(sector_weights)), 
    labels=collect(keys(sector_weights)),
    autopct="%1.1f%%")
title("Portfolio Sector Distribution")
savefig("sector_allocation.png")

Sector weights: Dict{Any, Any}(String31("Health Care") => 0.199878135356146, String31("Technology") => 0.1999684312135545, String31("Communication Services") => 0.20025012355521904, String31("Financial") => 0.19995663376783007, String31("Industrials") => 0.19994667610725037)
Total weight: 1.0


In [19]:
sorted_weights = sort(collect(zip(final_df.StockName, value.(x))), by=x->x[2], rev=true)

# Plot cumulative weights
figure(figsize=(12, 6))
weights = [w[2] for w in sorted_weights]
cumsum_weights = cumsum(weights)
plot(1:length(weights), cumsum_weights, "ro-")  # Changed 'ro-' to "ro-"
xticks(1:length(weights), [w[1] for w in sorted_weights], rotation=45, ha="right")
xlabel("Stocks")
ylabel("Cumulative Weight")
title("Portfolio Cumulative Weight Distribution")
grid(true)
tight_layout()
savefig("cumulative_weights.png")

## Trade-off Analysis Code

In [12]:
# Calculate risk and return for different λ values
λ_values = [1.0, 5.0, 10.0, 50.0, 100.0]
risk_return_pairs = []

for λ_test in λ_values
    m_test = Model(Ipopt.Optimizer)
    @variable(m_test, x_test[1:25] >= 0)
    @constraint(m_test, sum(x_test) == 1)
    @constraint(m_test, μ'*x_test >= R_min)
    @objective(m_test, Min, -μ'*x_test + λ_test * (x_test'*Σ*x_test))
    optimize!(m_test)
    
    if termination_status(m_test) in [MOI.OPTIMAL, MOI.LOCALLY_SOLVED]
        x_opt = value.(x_test)
        risk = x_opt'*Σ*x_opt
        ret = dot(μ, x_opt)
        push!(risk_return_pairs, (risk, ret, λ_test))
    end
end

# Plot risk-return curve
figure(figsize=(10, 6))
scatter([p[1] for p in risk_return_pairs], 
        [p[2] for p in risk_return_pairs], 
        c="blue", s=100)
for i in 1:length(risk_return_pairs)
    annotate("λ=$(risk_return_pairs[i][3])", 
             (risk_return_pairs[i][1], risk_return_pairs[i][2]))
end
xlabel("Portfolio Risk (Variance)")
ylabel("Expected Return (%)")
title("Risk-Return Trade-off Analysis")
grid(true)
savefig("risk_return_tradeoff.png")

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       25
Number of nonzeros in inequality constraint Jacobian.:       25
Number of nonzeros in Lagrangian Hessian.............:      325

Total number of variables............................:       25
                     variables with only lower bounds:       25
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -4.0664543e-02 7.50e-01 9.59e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

## Sensitivity Analysis Code

In [13]:

λ_values = [1.0, 5.0, 10.0, 50.0, 100.0]

R_min_values = [0.001, 1.0, 2.0, 3.0]


function optimize_portfolio(μ, Σ, R_min, λ)
    n = length(μ)
    m = Model(Ipopt.Optimizer)
    @variable(m, x[1:n] >= 0)
    @constraint(m, sum(x) == 1)
    @constraint(m, μ'*x >= R_min)
    @objective(m, Min, -μ'*x + λ * (x'*Σ*x))
    
    optimize!(m)
    return termination_status(m) in [MOI.OPTIMAL, MOI.LOCALLY_SOLVED] ? (value.(x), dot(μ, value.(x)), value.(x)'*Σ*value.(x)) : nothing
end


results = []
for R_min in R_min_values
    for λ in λ_values
        println("analysis R_min = $R_min, λ = $λ")
        result = optimize_portfolio(μ, Σ, R_min, λ)
        if !isnothing(result)
            push!(results, (R_min, λ, result[2], result[3]))
        end
    end
end

println("\nAnalysis result:")
for (R_min, λ, ret, risk) in results
    println("R_min = $R_min, λ = $λ: Return = $(round(ret, digits=4)), Risk = $(round(risk, digits=4))")
end

figure(figsize=(12, 5))

for R_min in R_min_values
    R_min_results = filter(r -> r[1] == R_min, results)
    plot([r[2] for r in R_min_results], 
         [r[3] for r in R_min_results], 
         "o-", 
         label="R_min = $R_min")
end

xlabel("Risk Aversion (λ)")
ylabel("Portfolio Return")
title("Return vs Risk Aversion")
legend()
grid(true)
savefig("sensitivity.png")

analysis R_min = 0.001, λ = 1.0
This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       25
Number of nonzeros in inequality constraint Jacobian.:       25
Number of nonzeros in Lagrangian Hessian.............:      325

Total number of variables............................:       25
                     variables with only lower bounds:       25
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -4.0664543e-02 7.50e-01 9.59e-01  -1.0 0

In [14]:
using JuMP, Ipopt, PyPlot, LinearAlgebra

figure(figsize=(15, 6))

colors = ["b", "r", "g", "m"]  

# return plot
subplot(1, 2, 1)
for (i, R_min) in enumerate(R_min_values)
    R_min_results = filter(r -> r[1] == R_min, results)
    plot([r[2] for r in R_min_results], 
         [r[3] for r in R_min_results], 
         "o-", 
         color=colors[i],
         label="R_min = $R_min")
end
xlabel("Risk Aversion (λ)")
ylabel("Portfolio Return")
title("Return vs Risk Aversion")
legend()
grid(true)

# risk plot
subplot(1, 2, 2)
for (i, R_min) in enumerate(R_min_values)
    R_min_results = filter(r -> r[1] == R_min, results)
    plot([r[2] for r in R_min_results], 
         [r[4] for r in R_min_results], 
         "o-", 
         color=colors[i],
         label="R_min = $R_min")
end
xlabel("Risk Aversion (λ)")
ylabel("Portfolio Risk")
title("Risk vs Risk Aversion")
legend()
grid(true)

tight_layout()
savefig("risk_return_tradeoff.png")

# 4. Results and Discussion

# 5. Conclusion

# 6. Appendix

# 7. References

- Team, T. I. (2023, August 29). Modern portfolio theory: What MPT is and how investors use it. Investopedia. https://www.investopedia.com/terms/m/modernportfoliotheory.asp?utm_source=chatgpt.com 